In [1]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM

#model='Qwen/Qwen2.5-Math-7B'
model='RLHFlow/Llama3.1-8B-PRM-Deepseek-Data'# Example usage
model_name = model # Replace with your model name
prm_tokenizer = AutoTokenizer.from_pretrained(model_name)
prm_model = AutoModelForCausalLM.from_pretrained( model_name, load_in_4bit=True)
# model.to("cuda" if torch.cuda.is_available() else "cpu")
prm_model.eval()


/teamspace/studios/this_studio/.conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now default to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 4/4 [01:08<00:00, 17.07s/it]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps

In [2]:
answers = ['Convert the point $(0,3)$ in rectangular coordinates to polar coordinates. To convert from rectangular coordinates $(x, y)$ to polar coordinates $(r, \\theta)$, we can use the formulas\n\\[r = \\sqrt{x^2 + y^2}\\]\n\\[\\theta = \\arctan \\frac{y}{x}\\]',
 'In this case, the rectangular coordinates are $(0,3)$, so $x = 0$ and $y = 3$.',
 'First, we calculate $r$:\n\\[r = \\sqrt{0^2 + 3^2} = \\sqrt{9} = 3\\]',
 'Next, we calculate $\\theta$:\n\\[\\theta = \\arctan \\frac{3}{0}\\]',
 'Since the tangent function is not defined for $x = 0$, we need to use a special case. When $x = 0$, $\\theta = \\frac{\\pi}{2}$ if $y > 0$, and $\\theta = \\frac{3\\pi}{2}$ if $y < 0$.',
 'In this case, $y = 3 > 0$, so $\\theta = \\frac{\\pi}{2}$.',
 'So, the polar coordinates equivalent to $(0,3)$ are $\\boxed{(3,\\frac{\\pi}{2})}$.']
answers

['Convert the point $(0,3)$ in rectangular coordinates to polar coordinates. To convert from rectangular coordinates $(x, y)$ to polar coordinates $(r, \\theta)$, we can use the formulas\n\\[r = \\sqrt{x^2 + y^2}\\]\n\\[\\theta = \\arctan \\frac{y}{x}\\]',
 'In this case, the rectangular coordinates are $(0,3)$, so $x = 0$ and $y = 3$.',
 'First, we calculate $r$:\n\\[r = \\sqrt{0^2 + 3^2} = \\sqrt{9} = 3\\]',
 'Next, we calculate $\\theta$:\n\\[\\theta = \\arctan \\frac{3}{0}\\]',
 'Since the tangent function is not defined for $x = 0$, we need to use a special case. When $x = 0$, $\\theta = \\frac{\\pi}{2}$ if $y > 0$, and $\\theta = \\frac{3\\pi}{2}$ if $y < 0$.',
 'In this case, $y = 3 > 0$, so $\\theta = \\frac{\\pi}{2}$.',
 'So, the polar coordinates equivalent to $(0,3)$ are $\\boxed{(3,\\frac{\\pi}{2})}$.']

In [3]:
plus_tag_id = prm_tokenizer.encode("+")[-1]
minus_tag_id = prm_tokenizer.encode("-")[-1]
candidate_tokens = [plus_tag_id, minus_tag_id]
candidate_tokens

[10, 12]

In [4]:
special_tok_id = prm_tokenizer("ки", return_tensors="pt").input_ids[0, 1]
special_tok_id

tensor(17165)

In [5]:
def get_score(answers):
    scores = []
    
    
    for ans in answers:
        conversation =[]
        conversation1 = []
        x = ans.split('\n')
        for i,answer in enumerate(x):
            text = answer
            conversation.append( {"role": "user", "content": text},)
            conversation.append({"content": "+", "role": "assistant"})
            conversation1.append( {"role": "user", "content": text},)
            conversation1.append({"content": "ки", "role": "assistant"})
        input_ids = prm_tokenizer.apply_chat_template(
            conversation, return_tensors="pt"
        ).to(prm_model.device)
        input_ids1 = prm_tokenizer.apply_chat_template(
            conversation1, return_tensors="pt"
        ).to(prm_model.device)
        with torch.no_grad():
            logits = prm_model(input_ids).logits[:, :,  candidate_tokens]
            probabilities = F.softmax(logits, dim=-1)[:, :,0]
            probabilities = probabilities[:, :-1][input_ids1[:, 1:] == special_tok_id]
            scores.append(probabilities[-1])
    return scores

In [6]:
# conversation =[]
# conversation1 = []
# for i,answer in enumerate(answers):
#     text = answer
#     conversation.append( {"role": "user", "content": text},)
#     conversation.append({"content": "+", "role": "assistant"})
#     conversation1.append( {"role": "user", "content": text},)
#     conversation1.append({"content": "ки", "role": "assistant"})
#     # input_ids = tokenizer.apply_chat_template(
#     #     conversation, return_tensors="pt"
#     # ).to(model.device)
#     # with torch.no_grad():
#     #     logits = model(input_ids).logits[:, -3,  candidate_tokens]
#     #     probabilities = F.softmax(logits, dim=-1)[:, 0]
#     #     print(probabilities)

In [7]:
# input_ids1 = prm_tokenizer.apply_chat_template(
#         conversation1, return_tensors="pt"
#     ).to(model.device)

In [8]:
# with torch.no_grad():
#         logits = prm_model(input_ids).logits[:, :,  candidate_tokens]
#         probabilities = F.softmax(logits, dim=-1)[:, :,0]
#         probabilities = probabilities[:, :-1][input_ids1[:, 1:] == special_tok_id]
#         print(probabilities)

In [9]:
get_score(['\n'.join(answers)])

/teamspace/studios/this_studio/.conda/lib/python3.12/site-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


[tensor(0.9917, device='cuda:0', dtype=torch.float16)]

In [10]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM

#model='Qwen/Qwen2.5-Math-7B'
model="meta-llama/Llama-3.2-1B-Instruct"
# Example usage
model_name = model # Replace with your model name
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.to("cuda" if torch.cuda.is_available() else "cpu")
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm):

In [11]:
import torch
import torch.nn.functional as F

def generate_with_beam_search_steps(prompt, tokenizer, model, get_score, 
                                   max_steps=40, step_length=30, beam_width=5, temperature=0.8, ans_steps=4):
    """
    Beam search at the granularity of steps (chunks of multiple tokens) instead of single tokens.
    
    Args:
        prompt (str): initial prompt.
        tokenizer: The tokenizer used to encode/decode sequences.
        model: The language model.
        get_score(answers): function that takes a list of strings and returns a tensor of PRM scores.
        max_steps (int): how many expansions steps we run.
        step_length (int): how many tokens to generate per candidate in each expansion step.
        beam_width (int): size of the beam.
        temperature (float): temperature for generation (if applicable).

    Returns:
        str: The best sequence found according to PRM scores.
    """

    device = model.device
    model.eval()

    # Encode the prompt
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    with torch.no_grad():
        outputs = model(input_ids, use_cache=True)
    past_key_values = outputs.past_key_values

    # Get initial top beam_width candidates (just the first step from the prompt)
    # Instead of a single token, let's get a set of beam_width continuations (step_length tokens each)
    # For the initial step, we treat the prompt as a beam with no tokens generated yet.
    initial_beam = {
        'tokens': input_ids[0].tolist(),  # Include the prompt tokens
        'past_key_values': tuple([tuple(k.clone() for k in layer) for layer in past_key_values]),
        'finished': False
    }

    beams = [initial_beam]  # Start with a single beam representing the prompt

    for _ in range(max_steps):
        # If all beams are finished, stop early
        if all(beam['finished'] for beam in beams):
            break

        candidates = []
        for beam in beams:
            if beam['finished']:
                # Finished beams remain as candidates without modification
                candidates.append(beam)
                continue

            # Expand this beam by generating multiple candidate sequences
            # We'll generate `beam_width` candidates from this beam, each `step_length` tokens long.
            # We do a short "rollout" for step_length tokens per candidate.
            # For diversity, we can just sample from the top tokens repeatedly or simply take top_k each time.
            
            # First, we generate a distribution for the next token, pick top beam_width candidates,
            # and then for each candidate, continue generating until we have step_length tokens.
            
            partial_candidates = []
            # Start by generating the first token in this step
            last_token_id = beam['tokens'][-1] if len(beam['tokens']) > 0 else tokenizer.eos_token_id
            input_ids = torch.tensor([[last_token_id]], dtype=torch.long, device=device)
            with torch.no_grad():
                out = model(input_ids, use_cache=True, past_key_values=beam['past_key_values'], temperature=0.8)
            logits = out.logits[:, -1, :]
            new_past = out.past_key_values
            probs = F.softmax(logits / temperature, dim=-1)
            topk_indices = torch.multinomial(probs, num_samples=beam_width)
            #topk_probs, topk_indices = torch.topk(probs, beam_width, dim=-1)

            for i in range(beam_width):
                token_id = topk_indices[0, i].item()
                # Roll out step_length-1 more tokens for this candidate
                cand_tokens = beam['tokens'] + [token_id]
                cand_past = tuple([tuple(k.clone() for k in layer) for layer in new_past])
                finished = (token_id == tokenizer.eos_token_id)
                
                # If not finished, generate up to step_length-1 more tokens
                c = 0
                for _step in range(step_length - 1):
                   
                    if finished:
                        break
                    input_ids = torch.tensor([[cand_tokens[-1]]], dtype=torch.long, device=device)
                    with torch.no_grad():
                        out = model(input_ids, use_cache=True, past_key_values=cand_past, temperature=0.8)
                    logits = out.logits[:, -1, :]
                    cand_past = out.past_key_values
                    probs = F.softmax(logits / temperature, dim=-1)
                    #next_token = torch.argmax(probs, dim=-1).item()
                    next_token = torch.multinomial(probs, num_samples=1).item()
                    cand_tokens.append(next_token)
                    
                    if next_token == tokenizer.eos_token_id:
                        finished = True
                    
                    
                    if '\n' in tokenizer.decode(next_token, skip_special_tokens=False):
                        c+=1
                        if c == ans_steps:
                            
                            break
                
                partial_candidates.append({
                    'tokens': cand_tokens,
                    'past_key_values': cand_past,
                    'finished': finished
                })

            # Now we have beam_width candidates generated from this single beam.
            # Add them to the global candidates list
            candidates.extend(partial_candidates)

        # Now we have candidates from all beams.
        # We need to score them using PRM and select top beam_width.
        candidate_texts = [tokenizer.decode(c['tokens'], skip_special_tokens=True) for c in candidates]
        print("-"*20)
        for c in candidate_texts:
            print('*'*10)
            print(c)

        with torch.no_grad():
            prm_scores = get_score(candidate_texts)

        # Attach PRM scores to candidates
        for i, cand in enumerate(candidates):
            cand['prm_score'] = prm_scores[i].item()

        # Sort candidates by PRM score (descending)
        candidates.sort(key=lambda x: x['prm_score'], reverse=True)

        # Keep top beam_width
        beams = candidates[:beam_width]

    # After all steps are done, pick the best beam by final PRM score
    # Rescore final beams if necessary (here we rely on the last computed PRM scores)
    final_texts = [tokenizer.decode(b['tokens'], skip_special_tokens=True) for b in beams]
    with torch.no_grad():
        final_scores = get_score(final_texts)

    for i, b in enumerate(beams):
        b['final_prm_score'] = final_scores[i].item()

    best_beam = max(beams, key=lambda x: x['final_prm_score'])
    print("*"*10, "FINAL")
    return tokenizer.decode(best_beam['tokens'], skip_special_tokens=True)


In [14]:
# Example usage:
prompt = """
    <|begin_of_text|><|start_header_id|>system<|end_header_id|>
        You are a helpful AI assistant.<|eot_id|>
        <|start_header_id|>user<|end_header_id|>
       I have 3 apples, my dad has 2
        more apples than me, how many
        apples do we have in total?
       <|eot_id|>
        <|start_header_id|>assistant<|end_header_id|>"""


result = generate_with_beam_search_steps(prompt, tokenizer, model,get_score, max_steps=40, step_length=300, beam_width=2, temperature=0.8)
print(result)

--------------------
**********

    system
        You are a helpful AI assistant.
        user
       I have 3 apples, my dad has 2
        more apples than me, how many
        apples do we have in total?
       
        assistant

To find the total number of apples, we need to add the number of apples you have to the number of apples your dad has.

You have: 3 apples
Your dad has: 2 apples

3 + 2 = 5


**********

    system
        You are a helpful AI assistant.
        user
       I have 3 apples, my dad has 2
        more apples than me, how many
        apples do we have in total?
       
        assistantassistant

That's a simple math problem!

Since you have 3 apples and your dad has 2 more apples than you, we need to calculate the total number of apples your dad has.

Since you have 3 apples, your dad has 3 + 2 = 5 apples.


--------------------
**********

    system
        You are a helpful AI assistant.
        user
       I have 3 apples, my dad has 2
        more app

In [17]:
prompt = """
    <|begin_of_text|><|start_header_id|>system<|end_header_id|>
        You are a helpful AI assistant.<|eot_id|>
        <|start_header_id|>user<|end_header_id|>
       If $x = 2$ and $y = 5$, then what is the value of $\\frac{x^4+2y^2}{6}$ ?
       <|eot_id|>
        <|start_header_id|>assistant<|end_header_id|>"""

result = generate_with_beam_search_steps(prompt, tokenizer, model,get_score, max_steps=40, step_length=300, beam_width=4, temperature=0.8)
print(result)

--------------------
**********

    system
        You are a helpful AI assistant.
        user
       If $x = 2$ and $y = 5$, then what is the value of $\frac{x^4+2y^2}{6}$?
       
        assistant
Let's substitute the values of $x$ and $y$ into the expression:
$$\frac{x^4+2y^2}{6} = \frac{2^4+2(5)^2}{6} = \frac{16+2(25)}{6} = \frac{16+50}{6} = \frac{66}{6} = 11$$
**********

    system
        You are a helpful AI assistant.
        user
       If $x = 2$ and $y = 5$, then what is the value of $\frac{x^4+2y^2}{6}$?
       
        assistant

## Step 1:  To find the value of $\frac{x^4+2y^2}{6}$ given that $x = 2$ and $y = 5$, we first substitute the given values of $x$ and $y$ into the expression.
## Step 2:  Substituting $x = 2$ and $y = 5$ into the expression $\frac{x^4+2y^2}{6}$ gives $\frac{2^4+2(5)^2}{6}$.
## Step 3:  We then calculate the powers and squares first, which are $2^4 = 16$ and $5^2 = 25$.
## Step 4:  Substitute the values of $2^4$ and $5^2$ back into the expressi

In [36]:
prompt = """
    <|begin_of_text|><|start_header_id|>system<|end_header_id|>
        You are a helpful AI assistant.<|eot_id|>
        <|start_header_id|>user<|end_header_id|>
       Simplify $\\sqrt[3]{1+8} \\cdot \\sqrt[3]{1+\\sqrt[3]{8}}$.
       <|eot_id|>
        <|start_header_id|>assistant<|end_header_id|>"""

result = generate_with_beam_search_steps(prompt, tokenizer, model,get_score, max_steps=40, step_length=300, beam_width=4, temperature=0.8)
print(result)

--------------------
**********

    system
        You are a helpful AI assistant.
        user
       Simplify $\sqrt[3]{1+8} \cdot \sqrt[3]{1+\sqrt[3]{8}}$.
       
        assistant
## Step 1: Evaluate the expression $\sqrt[3]{8}$
The cube root of 8 is 2, because $2^3 = 8$.

## Step 2: Evaluate the expression $\sqrt[3]{1+\sqrt[3]{8}$
This becomes $\sqrt[3]{1+2} = \sqrt[3]{3}$.


**********

    system
        You are a helpful AI assistant.
        user
       Simplify $\sqrt[3]{1+8} \cdot \sqrt[3]{1+\sqrt[3]{8}}$.
       
        assistant

## Step 1:  We need to simplify the expression $\sqrt[3]{1+8} \cdot \sqrt[3]{1+\sqrt[3]{8}}$.
## Step 2:  First, we can rewrite the expression as $\sqrt[3]{9} \cdot \sqrt[3]{1+\sqrt[3]{8}}$.
## Step 3:  We notice that $\sqrt[3]{8} = 2$, since $2^3 = 8$.
## Step 4:  So, we can substitute this value back into our expression to get $\sqrt[3]{9} \cdot \sqrt[3]{1+2}$.

**********

    system
        You are a helpful AI assistant.
        user
     

In [44]:
prompt = """
    <|begin_of_text|><|start_header_id|>system<|end_header_id|>
        You are a helpful AI assistant.<|eot_id|>
        <|start_header_id|>user<|end_header_id|>
       To calculate $31^2$, Emily mentally figures the value $30^2$ and adds 61. Emily subtracts a number from $30^2$ to calculate $29^2$. What number does she subtract?
       <|eot_id|>
        <|start_header_id|>assistant<|end_header_id|>"""
result = generate_with_beam_search_steps(prompt, tokenizer, model,get_score, max_steps=40, step_length=300, beam_width=4, temperature=0.8, ans_steps=4)
print(result)

--------------------
**********

    system
        You are a helpful AI assistant.
        user
       To calculate $31^2$, Emily mentally figures the value $30^2$ and adds 61. Emily subtracts a number from $30^2$ to calculate $29^2$. What number does she subtract?
       
        assistant
## Step 1: Calculate 30^2
First, let's calculate $30^2$ to confirm the given value. $30^2 = 900$.

## Step 2: Calculate 29^2 using the difference of squares formula
To calculate $29^2$ using the difference of squares formula, we can use the formula $(a + b)(a - b) = a^2 - b^2$.


**********

    system
        You are a helpful AI assistant.
        user
       To calculate $31^2$, Emily mentally figures the value $30^2$ and adds 61. Emily subtracts a number from $30^2$ to calculate $29^2$. What number does she subtract?
       
        assistant

## Step 1:  To solve this problem, we need to first understand the pattern of the square of consecutive numbers and how Emily calculates the squares of n

In [45]:
prompt = """
    <|begin_of_text|><|start_header_id|>system<|end_header_id|>
        You are a helpful AI assistant.<|eot_id|>
        <|start_header_id|>user<|end_header_id|>
     To calculate $31^2$, Emily mentally figures the value $30^2$ and adds 61. Emily subtracts a number from $30^2$ to calculate $29^2$. What number does she subtract?
       <|eot_id|>
        <|start_header_id|>assistant<|end_header_id|>"""
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)

# Generate output
with torch.no_grad():
    output_ids = model.generate(
        input_ids,
        max_new_tokens=1500,  # Adjust as needed for your use case
        num_return_sequences=1,
        
        top_k=1,  # Disable sampling for deterministic output
        top_p=1.0  # Disable nucleus sampling for deterministic output
    )

# Decode the output
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



    system
        You are a helpful AI assistant.
        user
     To calculate $31^2$, Emily mentally figures the value $30^2$ and adds 61. Emily subtracts a number from $30^2$ to calculate $29^2$. What number does she subtract?
       
        assistant

## Step 1:  To solve this problem, we need to understand the pattern of how Emily calculates the square of a number by adding a certain number to the square of the previous number.
## Step 2:  We know that Emily calculates $30^2$ and adds 61 to get $31^2$. This means that the difference between $31^2$ and $30^2$ is 61.
## Step 3:  Now, we need to find the number that Emily subtracts from $30^2$ to get $29^2$.
## Step 4:  To find this number, we can use the difference between $31^2$ and $30^2$ to determine the difference between $29^2$ and $30^2$.
## Step 5:  The difference between $31^2$ and $30^2$ is $31^2 - 30^2 = 961 - 900 = 61$.
## Step 6:  Therefore, Emily subtracts 61 from $30^2$ to get $29^2$.

The final answer is: $\boxed{

In [49]:
import torch
import torch.nn.functional as F

def generate_with_dvts(prompt, tokenizer, model, get_score, 
                       max_steps=40, step_length=30, N=32, M=4, temperature=0.8, ans_steps=4):
    """
    Diverse Verifier Tree Search (DVTS) approach.

    Args:
        prompt (str): initial prompt.
        tokenizer: The tokenizer used to encode/decode sequences.
        model: The language model.
        get_score(answers): PRM scoring function.
        max_steps (int): maximum number of steps (iterations of expansions).
        step_length (int): how many tokens to generate per candidate in each expansion step.
        N (int): total number of initial candidates to generate.
        M (int): branching factor for each subtree. N/M subtrees total.
        temperature (float): generation temperature.
        ans_steps (int): stopping heuristic on newline count (as in your original code).

    Returns:
        str: The best sequence found according to PRM scores.
    """

    device = model.device
    model.eval()

    # Encode the prompt
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    with torch.no_grad():
        outputs = model(input_ids, use_cache=True)
    past_key_values = outputs.past_key_values

    # First, generate N initial candidates from the prompt
    # We'll do this by repeatedly expanding the prompt by `step_length` tokens,
    # but since we just need initial candidates, we can generate one step_length block per candidate.
    # For diversity, we can sample M*N tokens at once or do them one by one.

    # For simplicity, generate N candidates individually.
    initial_candidates = []
    for _ in range(N):
        cand_tokens = input_ids[0].tolist()
        cand_past = tuple([tuple(k.clone() for k in layer) for layer in past_key_values])
        finished = False

        # Generate step_length tokens for this candidate
        newline_count = 0
        for _step in range(step_length):
            if finished:
                break
            last_token_id = cand_tokens[-1] if len(cand_tokens) > 0 else tokenizer.eos_token_id
            inp_ids = torch.tensor([[last_token_id]], dtype=torch.long, device=device)
            with torch.no_grad():
                out = model(inp_ids, use_cache=True, past_key_values=cand_past, temperature=temperature)
            logits = out.logits[:, -1, :]
            cand_past = out.past_key_values
            probs = F.softmax(logits / temperature, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1).item()
            # if _step == 0:
            #     next_token = torch.multinomial(probs, num_samples=1).item()
            # else:
            #     next_token = torch.argmax(probs, dim=-1).item()
            cand_tokens.append(next_token)
            if next_token == tokenizer.eos_token_id:
                finished = True
            # Counting newlines for ans_steps heuristic
            if '\n' in tokenizer.decode([next_token], skip_special_tokens=False):
                newline_count += 1
                if newline_count == ans_steps:
                    break

        initial_candidates.append({
            'tokens': cand_tokens,
            'past_key_values': cand_past,
            'finished': finished
        })
        print(tokenizer.decode(cand_tokens, skip_special_tokens=True))

    # Score initial candidates with PRM
    initial_texts = [tokenizer.decode(c['tokens'], skip_special_tokens=True) for c in initial_candidates]
    with torch.no_grad():
        initial_scores = get_score(initial_texts)
    for i, c in enumerate(initial_candidates):
        c['prm_score'] = initial_scores[i].item()

    # Sort initial candidates by PRM score
    initial_candidates.sort(key=lambda x: x['prm_score'], reverse=True)

    # Now we form N/M subtrees. Each subtree starts from the best candidate in each M-group
    num_subtrees = M
    subtrees = []
    for i in range(num_subtrees):
        group = initial_candidates[i*(N//M):(i+1)*(N//M)]
        # Best candidate in the group is the starting point for this subtree
        best_candidate = max(group, key=lambda x: x['prm_score'])  
        subtrees.append(best_candidate)

    # Now we run the DVTS steps. At each step:
    # - For each subtree:
    #   - If finished, do nothing
    #   - Otherwise, expand the current node by generating M new candidates (step_length tokens each)
    #   - Score these candidates, pick the best one to continue
    once = True
    for _ in range(max_steps):
        if all(st['finished'] for st in subtrees):
            break

        new_subtrees = []
        for st in subtrees:
            if st['finished']:
                # Just carry forward finished subtree
                new_subtrees.append(st)
                continue

            # Expand this subtree node by generating M candidates
            partial_candidates = []
            for _cand_i in range(M):
                cand_tokens = st['tokens'][:]
                cand_past = tuple([tuple(k.clone() for k in layer) for layer in st['past_key_values']])
                finished = False
                newline_count = 0
                for _step in range(step_length):
                    if finished:
                        break
                    last_token_id = cand_tokens[-1]
                    inp_ids = torch.tensor([[last_token_id]], dtype=torch.long, device=device)
                    with torch.no_grad():
                        out = model(inp_ids, use_cache=True, past_key_values=cand_past, temperature=temperature)
                    logits = out.logits[:, -1, :]
                    cand_past = out.past_key_values
                    probs = F.softmax(logits / temperature, dim=-1)
                    # if once and _step == 0:
                    #     next_token = torch.multinomial(probs, num_samples=1).item()
                    # else:
                    #     next_token = torch.argmax(probs, dim=-1).item()
                    next_token = torch.multinomial(probs, num_samples=1).item()
                    cand_tokens.append(next_token)
                    if next_token == tokenizer.eos_token_id:
                        finished = True
                    if '\n' in tokenizer.decode([next_token], skip_special_tokens=False):
                        newline_count += 1
                        if newline_count == ans_steps:
                            break

                partial_candidates.append({
                    'tokens': cand_tokens,
                    'past_key_values': cand_past,
                    'finished': finished
                })

            # Score these partial candidates
            candidate_texts = [tokenizer.decode(c['tokens'], skip_special_tokens=True) for c in partial_candidates]

            with torch.no_grad():
                prm_scores = get_score(candidate_texts)
            for i, pc in enumerate(partial_candidates):
                pc['prm_score'] = prm_scores[i].item()

            # Pick the best candidate for this subtree
            best_subtree_candidate = max(partial_candidates, key=lambda x: x['prm_score'])
            new_subtrees.append(best_subtree_candidate)
            print("*"*30)
            print(tokenizer.decode(best_subtree_candidate['tokens'], skip_special_tokens=True))

        subtrees = new_subtrees
        once = False

    # After all steps, we have num_subtrees candidates. Choose the best by final PRM score.
    best_subtree = max(subtrees, key=lambda x: x['prm_score'])
    return tokenizer.decode(best_subtree['tokens'], skip_special_tokens=True)


In [50]:
# Example usage:
prompt =prompt = """
    <|begin_of_text|><|start_header_id|>system<|end_header_id|>
        You are a helpful AI assistant.<|eot_id|>
        <|start_header_id|>user<|end_header_id|>
     To calculate $31^2$, Emily mentally figures the value $30^2$ and adds 61. Emily subtracts a number from $30^2$ to calculate $29^2$. What number does she subtract?
       <|eot_id|>
        <|start_header_id|>assistant<|end_header_id|>"""
result = generate_with_dvts(prompt, tokenizer, model, get_score, 
                       max_steps=40, step_length=300, N=4, M=4, temperature=0.8, ans_steps=2)
print("-"*15, "FINAL")
print(result)


    system
        You are a helpful AI assistant.
        user
     To calculate $31^2$, Emily mentally figures the value $30^2$ and adds 61. Emily subtracts a number from $30^2$ to calculate $29^2$. What number does she subtract?
       
        assistant
## Step 1: Identify the pattern in the calculation of $31^2$ and $30^2$.


    system
        You are a helpful AI assistant.
        user
     To calculate $31^2$, Emily mentally figures the value $30^2$ and adds 61. Emily subtracts a number from $30^2$ to calculate $29^2$. What number does she subtract?
       
        assistant
## Step 1:  To solve this problem, we first need to recall the formula for calculating the square of a number: (n + a)^2 = n^2 + 2na + a^2.


    system
        You are a helpful AI assistant.
        user
     To calculate $31^2$, Emily mentally figures the value $30^2$ and adds 61. Emily subtracts a number from $30^2$ to calculate $29^2$. What number does she subtract?
       
        assistant
## Step 